# OMARI HAMZA & hamzaoui thameur G2 
## Projet : Fruits detection

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

import os, shutil, pathlib


In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

tf.config.list_physical_devices('GPU')


# 1. Setup directories and cleaning data of our dataset

In [ ]:
# Setup the directory of training, test and validation data
data_dir_train = "C:/Users/msihamza/Desktop/projet/Data/train"
data_dir_validation = "C:/Users/msihamza/Desktop/projet/Data/valid"
data_dir_test = "C:/Users/msihamza/Desktop/projet/Data/test"

# Managing the extension of the files
image_exts = ['jpeg','jpg', 'bmp', 'png']



# 2. Load the data

In [ ]:
# Specify image dimensions
IMAGE_SHAPE = (150, 150)

# Load data
# Load data with data augmentation
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    shear_range=.2,
    zoom_range=.2,
    width_shift_range=.2,
    height_shift_range=.2,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    data_dir_train,
    shuffle=True,
    target_size=IMAGE_SHAPE,
    class_mode='categorical'
)

datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

valid_generator = datagen.flow_from_directory(
    data_dir_validation,
    shuffle=False,
    target_size=IMAGE_SHAPE,
    batch_size=32,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    data_dir_test,
    shuffle=True,
    target_size=IMAGE_SHAPE,
    batch_size=32,
    class_mode='categorical'
)

# Check names and number of classes
names_of_classes = sorted(os.listdir(data_dir_test))
print(f"\nNames of Classes : {names_of_classes}".format(names_of_classes))
number_of_classes = len(names_of_classes)

# 3. Explore data

In [ ]:
# Check number of images in each subfolder for each class

def count_files_in_subfolder(subfolder):
    
    for path in pathlib.Path(subfolder).iterdir():
        if path.is_dir():
            print("Class " + str(path.name) + ": " + \
                  str(len([name for name in os.listdir(path) \
            if os.path.isfile(os.path.join(path, name))])) + " files")

print("Train data:")
count_files_in_subfolder(os.path.join(data_dir_train))
print("\nTest data:")
count_files_in_subfolder(os.path.join(data_dir_test))
print("\nValidation data:")
count_files_in_subfolder(os.path.join(data_dir_validation))

# 4. CNN model

In [ ]:
# model object
model = Sequential()

# Add Layers
model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=[150, 150, 3]))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(2, 2))

# Flatten the feature map
model.add(Flatten())

# Add the fully connected layers
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))


# print the model summary
model.summary()

In [ ]:
# Compile and fit the model
early_stopping = keras.callbacks.EarlyStopping(patience=5) # Set up callbacks
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics='accuracy')

hist = model.fit(
    train_generator, 
    epochs=10, 
    verbose=1, 
    validation_data=valid_generator, 
    steps_per_epoch = 320//32, 
    validation_steps = 40//32, 
    callbacks=early_stopping
)

# 5. Plotting the error and accuracy

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 6. Evaluate

In [ ]:
model.evaluate(test_generator)

In [ ]:
# Evaluate the model on the validation set
evaluation = model.evaluate(valid_generator)
print("Validation Loss: {:.2f}".format(evaluation[0]))
print("Validation Accuracy: {:.2f}%".format(evaluation[1] * 100))

# Test the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss: {:.2f}".format(test_loss))
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

In [ ]:
# Testing the 
test_image_path = r'C:\Users\msihamza\Desktop\projet\Data\train\betterave\IMG_20230529_192446.jpg'

def generate_predictions(test_image_path, actual_label):
    
    # 1. Load and preprocess the image
    test_img = image.load_img(test_image_path, target_size=(150, 150))
    test_img_arr = image.img_to_array(test_img)/255.0
    test_img_input = test_img_arr.reshape((1, test_img_arr.shape[0], test_img_arr.shape[1], test_img_arr.shape[2]))

    # 2. Make Predictions
    predicted_label = np.argmax(model.predict(test_img_input))
    predicted_vegetable = names_of_classes[predicted_label]
    plt.figure(figsize=(4, 4))
    plt.imshow(test_img_arr)
    plt.title("Predicted Label: {}, Actual Label: {}".format(predicted_vegetable, actual_label))
    plt.grid()
    plt.axis('off')
    plt.show()

# call the function
generate_predictions(test_image_path, actual_label='betterave')

# 7. Save model

In [ ]:
model.save('Veg1.h5')

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


In [ ]:
predictions = model.predict(test_generator)

In [ ]:
plt.plot(hist.history['loss'], label='Training Loss')
plt.plot(hist.history['val_loss'], label='Validation Loss')
plt.title('Loss Curves')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(hist.history['accuracy'], label='Training Accuracy')
plt.plot(hist.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Curves')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
for images, labels in test_generator:
    predictions = model.predict(images)
    predicted_labels = np.argmax(predictions, axis=1)
    misclassified_indices = np.where(predicted_labels != np.argmax(labels, axis=1))[0]
    
    for index in misclassified_indices:
        img = images[index]
        true_label = np.argmax(labels[index])
        predicted_label = predicted_labels[index]
        plt.imshow(img)
        plt.title(f"True: {true_label}, Predicted: {predicted_label}")
        plt.axis('off')
        plt.show()
    
    break  # Show only the first batch of misclassified images

In [ ]:
# Load the trained model
model = load_model('C:/Users/msihamza/Desktop/projet/Veg1.h5')

# Load an image for prediction
image_path = 'C:/Users/msihamza/Desktop/projet/Data/test/oignon/IMG_20230529_191928.jpg'
img = image.load_img(image_path, target_size=IMAGE_SHAPE)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Preprocess the image
img_array = img_array / 255.0

# Make the prediction
predictions = model.predict(img_array)
predicted_label = np.argmax(predictions[0])

# Print the predicted label
print(f"Predicted Label: {names_of_classes[predicted_label]}")
